# Imports and installs

In [1]:
%cd ../

/


In [1]:
# Clone the repository
!git clone https://github.com/Qualcomm-AI-research/geometric-algebra-transformer.git

Cloning into 'geometric-algebra-transformer'...
remote: Enumerating objects: 369, done.
remote: Counting objects: 100% (369/369), done.
remote: Compressing objects: 100% (253/253), done.
remote: Total 369 (delta 171), reused 304 (delta 112), pack-reused 0 (from 0)
Receiving objects: 100% (369/369), 386.79 KiB | 12.09 MiB/s, done.
Resolving deltas: 100% (171/171), done.


In [2]:
# Navigate into the project directory
%cd geometric-algebra-transformer

/content/geometric-algebra-transformer


In [3]:
!pip install torch torchvision numpy mlflow hydra-core omegaconf xformers torch_ema e3nn torch_geometric

In [4]:
!pip uninstall dgl -y  # Remove any existing DGL installation
# !pip install dgl-cu118 dglgo -f https://data.dgl.ai/wheels/repo.html  # Install DGL for CUDA 11.8 (matching Colab environment)
!pip install  dgl -f https://data.dgl.ai/wheels/torch-2.1/repo.html

Looking in links: https://data.dgl.ai/wheels/torch-2.1/repo.html
  Using cached https://data.dgl.ai/wheels/torch-2.1/dgl-2.4.0-cp310-cp310-manylinux1_x86_64.whl (7.8 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.1.0+${cpu}.html

Looking in links: https://data.pyg.org/whl/torch-2.1.0+.html
  Using cached torch_scatter-2.1.2.tar.gz (108 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl size=3658842 sha256=03cd4699a7ed0ba8486597c1d8839c06ba20387d14d470260f38aa3776d0e52f
  Stored in directory: /root/.cache/pip/wheels/92/f1/2b/3b46d54b134259f58c8363568569053248040859b1a145b3ce
Successfully built torch-scatter


Code to deal with geometric algebra:

In [ ]:
import functools
import itertools
import operator

import torch


# copied from the itertools docs
def _powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(len(s) + 1))


class ShortLexBasisBladeOrder:
    def __init__(self, n_vectors):
        self.index_to_bitmap = torch.empty(2**n_vectors, dtype=int)
        self.grades = torch.empty(2**n_vectors, dtype=int)
        self.bitmap_to_index = torch.empty(2**n_vectors, dtype=int)

        for i, t in enumerate(_powerset([1 << i for i in range(n_vectors)])):
            bitmap = functools.reduce(operator.or_, t, 0)
            self.index_to_bitmap[i] = bitmap
            self.grades[i] = len(t)
            self.bitmap_to_index[bitmap] = i
            del t  # enables an optimization inside itertools.combinations


def set_bit_indices(x: int):
    """Iterate over the indices of bits set to 1 in `x`, in ascending order"""
    n = 0
    while x > 0:
        if x & 1:
            yield n
        x = x >> 1
        n = n + 1


def count_set_bits(bitmap: int) -> int:
    """Counts the number of bits set to 1 in bitmap"""
    count = 0
    for i in set_bit_indices(bitmap):
        count += 1
    return count


def canonical_reordering_sign_euclidean(bitmap_a, bitmap_b):
    """
    Computes the sign for the product of bitmap_a and bitmap_b
    assuming a euclidean metric
    """
    a = bitmap_a >> 1
    sum_value = 0
    while a != 0:
        sum_value = sum_value + count_set_bits(a & bitmap_b)
        a = a >> 1
    if (sum_value & 1) == 0:
        return 1
    else:
        return -1


def canonical_reordering_sign(bitmap_a, bitmap_b, metric):
    """
    Computes the sign for the product of bitmap_a and bitmap_b
    given the supplied metric
    """
    bitmap = bitmap_a & bitmap_b
    output_sign = canonical_reordering_sign_euclidean(bitmap_a, bitmap_b)
    i = 0
    while bitmap != 0:
        if (bitmap & 1) != 0:
            output_sign *= metric[i]
        i = i + 1
        bitmap = bitmap >> 1
    return output_sign


def gmt_element(bitmap_a, bitmap_b, sig_array):
    """
    Element of the geometric multiplication table given blades a, b.
    The implementation used here is described in :cite:`ga4cs` chapter 19.
    """
    output_sign = canonical_reordering_sign(bitmap_a, bitmap_b, sig_array)
    output_bitmap = bitmap_a ^ bitmap_b
    return output_bitmap, output_sign


def construct_gmt(index_to_bitmap, bitmap_to_index, signature):
    n = len(index_to_bitmap)
    array_length = int(n * n)
    coords = torch.zeros((3, array_length), dtype=torch.uint8)
    k_list = coords[0, :]
    l_list = coords[1, :]
    m_list = coords[2, :]

    # use as small a type as possible to minimize type promotion
    mult_table_vals = torch.zeros(array_length)

    for i in range(n):
        bitmap_i = index_to_bitmap[i]

        for j in range(n):
            bitmap_j = index_to_bitmap[j]
            bitmap_v, mul = gmt_element(bitmap_i, bitmap_j, signature)
            v = bitmap_to_index[bitmap_v]

            list_ind = i * n + j
            k_list[list_ind] = i
            l_list[list_ind] = v
            m_list[list_ind] = j

            mult_table_vals[list_ind] = mul

    return torch.sparse_coo_tensor(indices=coords, values=mult_table_vals, size=(n, n, n))

In [ ]:
from typing import Tuple
import torch



class CliffordAlgebra:
    def __init__(self, metric):
        self.metric = torch.as_tensor(metric, dtype=torch.float32)
        self.num_bases = len(metric)
        self.bbo = ShortLexBasisBladeOrder(self.num_bases)
        self.dim = len(self.metric)
        self.n_blades = len(self.bbo.grades)
        self.cayley = construct_gmt(self.bbo.index_to_bitmap, self.bbo.bitmap_to_index, self.metric).to_dense()

    def to(self, device: torch.device):
        self.bbo.grades = self.bbo.grades.to(device)
        self.metric = self.metric.to(device)
        self.cayley = self.cayley.to(device)
        return self

    def geometric_product(self, a, b):
        return torch.einsum(
            "bfi,ijk,bfk->bfj",
            a.view(len(a), -1, len(self.cayley)),
            self.cayley,
            b.view(len(b), -1, len(self.cayley)),
        ).view(*a.size())

    def reverse(self, mv, blades=None):
        """Perform the reversion operation on multivectors, an operation specific to geometric algebra.

        In Geometric Algebra, the reverse of a multivector is formed by reversing the order of the vectors in each blade.

        Args:
            mv (torch.Tensor): Input multivectors.
            blades (Union[tuple, list, torch.Tensor], optional): Specify which blades are present in the multivector.

        Returns:
            torch.Tensor: The reversed multivector.
        """
        grades = self.bbo.grades.to(mv.device)
        if blades is not None:
            grades = grades[torch.as_tensor(blades, dtype=int)]
        signs = torch.pow(-1, torch.floor(grades * (grades - 1) / 2))
        return signs * mv.clone()

    def embed(self, tensor: torch.Tensor, tensor_index: Tuple[int]) -> torch.Tensor:
        """
        Embeds the input tensor into a multivector.

        This method takes a tensor and embeds it into a multivector, with the tensor elements assigned to the basis blades
        specified by the tensor_index.

        Args:
            tensor (torch.Tensor): The input tensor to be embedded.
            tensor_index (tuple[int]): A tuple of integers specifying the blades where tensor elements are to be placed.

        Returns:
            torch.Tensor: The multivector embedding of the input tensor.

        Raises:
            AssertionError: If the last dimension of tensor does not match the length of tensor_index.
        """
        assert tensor.size(-1) == len(tensor_index)
        blades = tuple(range(self.n_blades))
        mv = torch.zeros(*tensor.shape[:-1], len(blades), device=tensor.device)
        tensor_index = torch.as_tensor(tensor_index, device=tensor.device, dtype=int)
        shape = *(1,) * (mv.dim() - 1), -1
        tensor_index = tensor_index.view(shape).repeat(*tensor.size()[:-1], 1)
        mv = torch.scatter(mv, -1, tensor_index, tensor)
        return mv

    def get(self, mv: torch.Tensor, blade_index: Tuple[int]) -> torch.Tensor:
        """
        Extracts the components of a multivector corresponding to the specified blade indices.

        This method takes a multivector and a tuple of blade indices, and returns the components of the multivector
        that correspond to those indices.

        Args:
            mv (torch.Tensor): The input multivector.
            blade_index (tuple[int]): A tuple of integers specifying the blades to be extracted.

        Returns:
            torch.Tensor: The tensor of components corresponding to the specified blade indices.
        """
        blade_index = tuple(blade_index)
        return mv[..., blade_index]

    def mag2(self, mv):
        return self.geometric_product(self.reverse(mv), mv)

    def norm(self, mv):
        mag2 = self.mag2(mv)[..., :1]
        mag2.abs_().sqrt_()
        return mag2

    def sandwich(self, a, b):
        """aba'"""
        return self.geometric_product(self.geometric_product(a, b), self.reverse(a))

In [ ]:
import math

import torch
from torch import nn
from torch.nn.modules.utils import _pair



class PGAConjugateLinear(nn.Module):
    """
    Linear layer that applies the PGA conjugation to the input.

    Args:
        in_features (int): Number of input features.
        out_features (int): Number of output features.
        algebra (Algebra): Algebra object that defines the geometric product.
        input_blades (tuple): Nonnegative blades of the input multivectors.
        action_blades (tuple, optional): Blades of the action. Defaults to (0, 5, 6, 7, 8, 9, 10, 15),
                                         which encodes rotation and translation.
    """

    def __init__(
        self,
        in_features,
        out_features,
        algebra,
        input_blades,
        action_blades=(0, 5, 6, 7, 8, 9, 10, 15),
    ):
        super().__init__()
        assert torch.all(algebra.metric == torch.tensor([0, 1, 1, 1]))
        self.input_blades = input_blades
        self.in_features = in_features
        self.out_features = out_features
        self.algebra = algebra
        self.action_blades = action_blades
        self.n_action_blades = len(action_blades)
        self._action = nn.Parameter(torch.empty(out_features, in_features, self.n_action_blades))
        self.weight = nn.Parameter(torch.empty(out_features, in_features))
        self.embed_e0 = nn.Parameter(torch.zeros(in_features, 1))

        self.inverse = algebra.reverse

        self.reset_parameters()

    def reset_parameters(self):
        # Init the rotation parts uniformly.
        torch.nn.init.uniform_(self._action[..., 0], -1, 1)
        torch.nn.init.uniform_(self._action[..., 4:7], -1, 1)

        # Init the translation parts with zeros.
        torch.nn.init.zeros_(self._action[..., 1:4])
        torch.nn.init.zeros_(self._action[..., 7])

        norm = self.algebra.norm(self.algebra.embed(self._action.data, self.action_blades))
        assert torch.allclose(norm[..., 1:], torch.tensor(0.0), atol=1e-3)
        norm = norm[..., :1]
        self._action.data = self._action.data / norm

        torch.nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5))

    @property
    def action(self):
        return self.algebra.embed(self._action, self.action_blades)

    def forward(self, input):
        M = self.algebra.cayley
        k = self.action
        k_ = self.inverse(k)
        x = self.algebra.embed(input, self.input_blades)
        x[..., 14:15] = self.embed_e0
        # x[..., 14:15] = 1

        k_l = get_clifford_left_kernel(M, k, flatten=False)
        k_r = get_clifford_right_kernel(M, k_, flatten=False)

        x = torch.einsum("oi,poqi,qori,bir->bop", self.weight, k_r, k_l, x)

        x = self.algebra.get(x, self.input_blades)

        return x


class MultiVectorAct(nn.Module):
    """
    A module to apply multivector activations to the input.

    Args:
        channels (int): Number of channels in the input.
        algebra: The algebra object that defines the geometric product.
        input_blades (list, tuple): The nonnegative input blades.
        kernel_blades (list, tuple, optional): The blades that will be used to compute the activation. Defaults to all input blades.
        agg (str, optional): The aggregation method to be used. Options include "linear", "sum", and "mean". Defaults to "linear".
    """

    def __init__(self, channels, algebra, input_blades, kernel_blades=None, agg="linear"):
        super().__init__()
        self.algebra = algebra
        self.input_blades = tuple(input_blades)
        if kernel_blades is not None:
            self.kernel_blades = tuple(kernel_blades)
        else:
            self.kernel_blades = self.input_blades

        if agg == "linear":
            self.conv = nn.Conv1d(channels, channels, kernel_size=len(self.kernel_blades), groups=channels)
        self.agg = agg

    def forward(self, input):
        v = self.algebra.embed(input, self.input_blades)
        if self.agg == "linear":
            v = v * torch.sigmoid(self.conv(v[..., self.kernel_blades]))
        elif self.agg == "sum":
            v = v * torch.sigmoid(v[..., self.kernel_blades].sum(dim=-1, keepdim=True))
        elif self.agg == "mean":
            v = v * torch.sigmoid(v[..., self.kernel_blades].mean(dim=-1, keepdim=True))
        else:
            raise ValueError(f"Aggregation {self.agg} not implemented.")
        v = self.algebra.get(v, self.input_blades)
        return v

In [ ]:
import sys
sys.path.append('/content/geometric-algebra-transformer')

In [ ]:
!ls /content/geometric-algebra-transformer

CHANGELOG.md  docker  img      pyproject.toml  scripts	 tests
config	      gatr    LICENSE  README.md       setup.py  tests_regression


In [ ]:
%cd ../

/content


# N-body experiment

## Create dataset

In [12]:
# Define the base directory for saving data
BASEDIR = "/content/nbody_data"

# Modify PYTHONPATH to include the current directory
import os
os.environ['PYTHONPATH'] += ":/content/geometric-algebra-transformer"


# Generate the N-body dataset
!python scripts/generate_nbody_dataset.py base_dir="${BASEDIR}" seed=42


/content/geometric-algebra-transformer/gatr/primitives/bilinear.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sparse_basis = torch.load(filename).to(torch.float32)
Cr

## 1000 training samples

### GATr

In [13]:
# Run the training experiment with the GATr model
!python scripts/nbody_experiment.py base_dir="${BASEDIR}" seed=42 model=gatr_nbody data.subsample=0.01 training.steps=5000 run_name=gatr

/content/geometric-algebra-transformer/gatr/primitives/bilinear.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sparse_basis = torch.load(filename).to(torch.float32)
[2

In [26]:
!python scripts/nbody_experiment.py base_dir="${BASEDIR}" seed=42 model=gatr_nbody data.subsample=0.01 training.steps=10000 run_name=gatr_10000

/content/geometric-algebra-transformer/gatr/primitives/bilinear.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sparse_basis = torch.load(filename).to(torch.float32)
[2

In [ ]:
# !python scripts/nbody_experiment.py base_dir="${BASEDIR}" seed=42 model=gatr_nbody data.subsample=0.01 training.steps=50000 run_name=gatr_50000

### MLP

In [17]:
!python scripts/nbody_experiment.py base_dir="${BASEDIR}" seed=42 model=mlp_nbody data.subsample=0.01 training.steps=5000 run_name=mlp

/content/geometric-algebra-transformer/gatr/primitives/bilinear.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sparse_basis = torch.load(filename).to(torch.float32)
[2

In [22]:
!python scripts/nbody_experiment.py base_dir="${BASEDIR}" seed=42 model=mlp_nbody data.subsample=0.01 training.steps=20000 run_name=mlp_20000

/content/geometric-algebra-transformer/gatr/primitives/bilinear.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sparse_basis = torch.load(filename).to(torch.float32)
[2

In [18]:
!python scripts/nbody_experiment.py base_dir="${BASEDIR}" seed=42 model=mlp_nbody data.subsample=0.01 training.steps=50000 run_name=mlp_50000

/content/geometric-algebra-transformer/gatr/primitives/bilinear.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sparse_basis = torch.load(filename).to(torch.float32)
[2

### Ordinary transformer

In [19]:
!python scripts/nbody_experiment.py base_dir="${BASEDIR}" seed=42 model=transformer_nbody data.subsample=0.01 training.steps=5000 run_name=transformer

/content/geometric-algebra-transformer/gatr/primitives/bilinear.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sparse_basis = torch.load(filename).to(torch.float32)
[2

In [20]:
!python scripts/nbody_experiment.py base_dir="${BASEDIR}" seed=42 model=transformer_nbody data.subsample=0.01 training.steps=20000 run_name=transformer_20000

/content/geometric-algebra-transformer/gatr/primitives/bilinear.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sparse_basis = torch.load(filename).to(torch.float32)
[2

In [21]:
!python scripts/nbody_experiment.py base_dir="${BASEDIR}" seed=42 model=transformer_nbody data.subsample=0.01 training.steps=50000 run_name=transformer_50000

/content/geometric-algebra-transformer/gatr/primitives/bilinear.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sparse_basis = torch.load(filename).to(torch.float32)
[2

## 100 training samples

### GATr

In [28]:
!python scripts/nbody_experiment.py base_dir="${BASEDIR}" seed=42 model=gatr_nbody data.subsample=0.001 training.steps=5000 run_name=gatr100_5000

/content/geometric-algebra-transformer/gatr/primitives/bilinear.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sparse_basis = torch.load(filename).to(torch.float32)
[2

In [35]:
!python scripts/nbody_experiment.py base_dir="${BASEDIR}" seed=42 model=gatr_nbody data.subsample=0.001 training.steps=10000 run_name=gatr100_10000

/content/geometric-algebra-transformer/gatr/primitives/bilinear.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sparse_basis = torch.load(filename).to(torch.float32)
[2

In [ ]:
# !python scripts/nbody_experiment.py base_dir="${BASEDIR}" seed=42 model=gatr_nbody data.subsample=0.001 training.steps=50000 run_name=gatr100_50000

### MLP

In [29]:
!python scripts/nbody_experiment.py base_dir="${BASEDIR}" seed=42 model=mlp_nbody data.subsample=0.001 training.steps=5000 run_name=mlp100_5000

/content/geometric-algebra-transformer/gatr/primitives/bilinear.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sparse_basis = torch.load(filename).to(torch.float32)
[2

In [30]:
!python scripts/nbody_experiment.py base_dir="${BASEDIR}" seed=42 model=mlp_nbody data.subsample=0.001 training.steps=20000 run_name=mlp100_20000

/content/geometric-algebra-transformer/gatr/primitives/bilinear.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sparse_basis = torch.load(filename).to(torch.float32)
[2

In [31]:
!python scripts/nbody_experiment.py base_dir="${BASEDIR}" seed=42 model=mlp_nbody data.subsample=0.001 training.steps=50000 run_name=mlp100_50000

/content/geometric-algebra-transformer/gatr/primitives/bilinear.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sparse_basis = torch.load(filename).to(torch.float32)
[2

### Ordinary transformer

In [32]:
!python scripts/nbody_experiment.py base_dir="${BASEDIR}" seed=42 model=transformer_nbody data.subsample=0.001 training.steps=5000 run_name=transformer100_5000

/content/geometric-algebra-transformer/gatr/primitives/bilinear.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sparse_basis = torch.load(filename).to(torch.float32)
[2

In [33]:
!python scripts/nbody_experiment.py base_dir="${BASEDIR}" seed=42 model=transformer_nbody data.subsample=0.001 training.steps=20000 run_name=transformer100_20000

/content/geometric-algebra-transformer/gatr/primitives/bilinear.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sparse_basis = torch.load(filename).to(torch.float32)
[2

In [34]:
!python scripts/nbody_experiment.py base_dir="${BASEDIR}" seed=42 model=transformer_nbody data.subsample=0.001 training.steps=50000 run_name=transformer100_50000

/content/geometric-algebra-transformer/gatr/primitives/bilinear.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sparse_basis = torch.load(filename).to(torch.float32)
[2